In [1]:
import pandas as pd
import numpy as np

In [2]:
cleaned_train =pd.read_csv('cleaned_train.csv')
cleaned_test =pd.read_csv('cleaned_test.csv')

In [3]:
# There are 6 group fold, so do a train test on the 6 fold
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  auc, roc_auc_score, roc_curve

from sklearn.pipeline import make_pipeline

In [4]:
cv_gen =  GroupKFold(n_splits=6).split(cleaned_train, cleaned_train.IsDefaulted, groups=cleaned_train.TransactionStartTimeMonth)
cv = list(cv_gen)

In [5]:
#right features for modelling
train_col = [
              'CustomerId_ProductId_ct',
              'meanTransactionPerLoan',
              'ext_data_Value_CustomerId_mean',
              'ext_data_Value_CustomerId_max',
              'ext_data_Value_CustomerId_std',
              'ext_data_CustomerId_ProductCategory_Value_min',
              'TransactionStartTimeWeek',
              'Issue_Trans_Diff',
              'DueDateIs_month_start',
              'kmeans'
            ]


In [6]:
from sklearn.ensemble import RandomForestClassifier

C:\Users\adegunle\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
oof = np.zeros(len(cleaned_train))
Target_name="IsDefaulted"
not_used_cols=[Target_name,'TransactionStartTime']
features_name=[ f for f in train_col if f not in not_used_cols]
skf = GroupKFold(n_splits=6)
for i, (idxT, idxV) in enumerate( skf.split(cleaned_train, cleaned_train.IsDefaulted, groups=cleaned_train.TransactionStartTimeMonth) ):

    
    
    log_train = cleaned_train.loc[idxT,features_name]
    log_test = cleaned_train.loc[idxV,features_name]


    log_train_y = cleaned_train['IsDefaulted'][idxT]
    log_test_y = cleaned_train['IsDefaulted'][idxV]
 

    lr = RandomForestClassifier(random_state=2001, n_estimators=2000, max_depth=2,class_weight='balanced',min_samples_split=20,min_samples_leaf=20)

    lr.fit(log_train,log_train_y)

    pred = lr.predict_proba(log_test)[:,1]
    oof[idxV] = pred

    print('Fold ', i)
    print (roc_auc_score(log_test_y, pred))
print(features_name)
print(roc_auc_score(cleaned_train['IsDefaulted'], oof))    

Fold  0
0.9189427722036417
Fold  1
0.8156249999999999
Fold  2
0.9201680672268908
Fold  3
0.9680288461538461
Fold  4
0.8364779874213837
Fold  5
0.7857142857142857
['CustomerId_ProductId_ct', 'meanTransactionPerLoan', 'ext_data_Value_CustomerId_mean', 'ext_data_Value_CustomerId_max', 'ext_data_Value_CustomerId_std', 'ext_data_CustomerId_ProductCategory_Value_min', 'TransactionStartTimeWeek', 'Issue_Trans_Diff', 'DueDateIs_month_start', 'kmeans']
0.9148551196271901


In [8]:
from sklearn.ensemble import  RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  AdaBoostClassifier

In [9]:
model =  LogisticRegression(random_state=2001)

clf4 = AdaBoostClassifier(base_estimator =model, n_estimators=2000, learning_rate =0.2)

In [10]:
clf4.fit(cleaned_train[features_name], cleaned_train.IsDefaulted)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2001, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=0.2, n_estimators=2000, random_state=None)

In [11]:
test = pd.read_csv('Test.csv',parse_dates=['TransactionStartTime','IssuedDateLoan'])

In [12]:
test['IsDefaulted'] = np.nan

In [13]:
test.loc[cleaned_test.index, 'IsDefaulted']= clf4.predict_proba(cleaned_test[features_name])[:,1].round(3)

In [14]:
pd.DataFrame({"TransactionId": test.TransactionId.values, 'IsDefaulted':test.IsDefaulted}).to_csv("ada.csv", index=False)

In [15]:
predict =pd.read_csv('ada.csv');predict.head()

,TransactionId,IsDefaulted
0,TransactionId_925,0.492
1,TransactionId_1080,0.495
2,TransactionId_2315,0.496
3,TransactionId_1466,0.496
4,TransactionId_337,0.496
